In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/year.csv')


In [ ]:
! pip install lightgbm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
df2 = df.drop(['馬名','レースコード','タイム','着差','通過','騎手','人気','上がり'],axis = 'columns')
df2 = df2[df2['周り']!='芝']

In [ ]:
df2 = df2[~(df2['着順'].astype(str).str.contains('\D'))]
df2 = df2[~(df2['齢'].astype(str).str.contains('\D'))]
df2 = df2[~(df2['距離'].astype(str).str.contains('\D'))]

In [ ]:
df2['着順'] = df2['着順'].astype(int)
df2['齢'] = df2['齢'].astype(int)
df2['馬体重'] = df2['馬体重'].astype(int)
df2['距離'] = df2['距離'].astype(float)
df2['単勝'] = df2['単勝'].astype(float)

In [ ]:
#障害レース0、新馬～オープンが1～6
df2.loc[df2['クラス'].str.contains('障害',na=False), ['クラス']]=0
df2.loc[df2['クラス'].str.contains('新馬',na=False), ['クラス']]=1
df2.loc[df2['クラス'].str.contains('未勝利',na=False), ['クラス']]=2
df2.loc[df2['クラス'].str.contains('1勝',na=False), ['クラス']]=3
df2.loc[df2['クラス'].str.contains('2勝',na=False), ['クラス']]=4
df2.loc[df2['クラス'].str.contains('3勝',na=False), ['クラス']]=5
df2.loc[df2['クラス'].str.contains('オープン',na=False), ['クラス']]=6
df2['クラス'] = df2['クラス'].astype(int)

In [ ]:
#牡0、牝1、セン2
df2.loc[df2['性'].str.contains('牡',na=False), ['性']]=0
df2.loc[df2['性'].str.contains('牝',na=False), ['性']]=1
df2.loc[df2['性'].str.contains('セ',na=False), ['性']]=2
df2['性'] = df2['性'].astype(int)

In [ ]:
#ダート0、芝1、障害2
df2.loc[df2['コース'].str.contains('ダ',na=False), ['コース']]=0
df2.loc[df2['コース'].str.contains('芝',na=False), ['コース']]=1
df2['コース'] = df2['コース'].astype(int)

In [ ]:
#右回り0、左回り1
df2.loc[df2['周り'].str.contains('右',na=False), ['周り']]=0
df2.loc[df2['周り'].str.contains('左',na=False), ['周り']]=1
df2['周り'] = df2['周り'].astype(int)

In [ ]:
df2['馬場状態'].value_counts()
df2.loc[df2['馬場状態'].str.contains('良',na=False), ['馬場状態']]=0
df2.loc[df2['馬場状態'].str.contains('稍',na=False), ['馬場状態']]=1
df2.loc[df2['馬場状態'].str.contains('重',na=False), ['馬場状態']]=2
df2.loc[df2['馬場状態'].str.contains('不',na=False), ['馬場状態']]=3
df2['馬場状態'] = df2['馬場状態'].astype(int)

In [ ]:
df2.dtypes

着順        int64
枠番        int64
馬番        int64
性         int64
齢         int64
斤量      float64
単勝      float64
馬体重       int64
増減      float64
クラス       int64
コース       int64
周り        int64
距離      float64
馬場状態      int64
dtype: object

In [ ]:
clip_rank = lambda x: x if x<2 else 2
df2['着順'] = df2['着順'].map(clip_rank)
df2['着順'].value_counts()

2    37959
1     2963
Name: 着順, dtype: int64

In [ ]:
#データ切り分け
from sklearn.model_selection import train_test_split

X = df2.drop(['着順'],axis=1)
y = df2['着順']

X_train,X_test,y_train,y_test = train_test_split(X,y,stratify=y,test_size=0.3,random_state=0)
odds = X_test['単勝']

In [ ]:
import lightgbm as lgb
model = lgb.LGBMClassifier(boosting_type='goss', max_depth=5, random_state=0)

In [ ]:
from sklearn import metrics
model.fit(X_train, y_train)
#X_testに予想したいレースの情報を入れる
y_pred = model.predict(X_test)
y_pred_prob = model.predict_proba(X_test)
df_pred = pd.DataFrame({'target':y_test,'target_pred':y_pred})
df_pred_prob = pd.DataFrame({'odds':odds,'target':y_test,'rank1':y_pred_prob[:,0], 'rank2':y_pred_prob[:,1]})
df_pred_prob['ex'] = df_pred_prob['odds'] * df_pred_prob['rank1']
df_pred_prob

,odds,target,rank1,rank2,ex
31918,269.7,2,0.000317,0.999683,0.085571
40945,86.2,2,0.002908,0.997092,0.250698
9947,10.5,2,0.118520,0.881480,1.244462
44846,3.9,2,0.189302,0.810698,0.738276
13758,80.5,2,0.007921,0.992079,0.637628
...,...,...,...,...,...
39712,7.2,2,0.079561,0.920439,0.572837
1606,40.7,2,0.002173,0.997827,0.088431
3176,14.4,2,0.073785,0.926215,1.062502
32906,4.0,2,0.139347,0.860653,0.557387


In [ ]:
buy = df_pred_prob[df_pred_prob['ex'] > 1]
buy

,odds,target,rank1,rank2,ex
9947,10.5,2,0.118520,0.881480,1.244462
45577,20.6,2,0.054157,0.945843,1.115637
44231,8.4,1,0.136159,0.863841,1.143735
33404,6.4,2,0.158973,0.841027,1.017430
30518,14.4,2,0.076572,0.923428,1.102630
...,...,...,...,...,...
1434,31.1,2,0.060853,0.939147,1.892534
5465,10.9,2,0.122328,0.877672,1.333377
24916,68.7,2,0.014884,0.985116,1.022564
2593,24.7,2,0.052079,0.947921,1.286361


In [ ]:
buy[buy['target']==1].sum()

odds      1799.500000
target     133.000000
rank1       31.692868
rank2      101.307132
ex         178.134437
dtype: float64

In [ ]:
X_test

,枠番,馬番,性,齢,斤量,単勝,馬体重,増減,クラス,コース,周り,距離,馬場状態
31918,7,13,2,3,56.0,269.7,428,-8.0,2,1,1,1200.0,0
40945,4,7,0,5,55.0,86.2,478,-8.0,6,1,0,2000.0,0
9947,5,9,2,5,57.0,10.5,484,18.0,3,1,1,1200.0,0
44846,5,5,0,4,56.0,3.9,498,-2.0,3,1,0,2600.0,0
13758,7,11,2,6,55.0,80.5,440,0.0,5,1,1,2000.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
39712,5,9,1,5,55.0,7.2,518,0.0,5,0,0,1800.0,0
1606,5,8,1,4,52.0,40.7,424,22.0,3,0,0,1700.0,0
3176,5,6,1,3,52.0,14.4,478,2.0,3,1,0,1200.0,0
32906,5,5,0,2,51.0,4.0,454,-6.0,2,0,1,1800.0,0
